In [249]:
import numpy as np
import pandas as pd
import psycopg2
from tqdm import tqdm
tqdm.pandas()

In [250]:
business_raw = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')
business_raw.index +=1

In [ ]:
business_raw.head()

In [ ]:
business_raw.shape

In [ ]:
business_raw[business_raw["address"].isna()].shape

In [ ]:
business_raw[business_raw["attributes"].isna()].shape

In [ ]:
business_raw[business_raw["business_id"].isna()].shape

In [ ]:
business_raw[business_raw["categories"].isna()].shape

In [ ]:
business_raw[business_raw["city"].isna()]

In [ ]:
business_raw[business_raw["hours"].isna()].shape

In [ ]:
business_raw[business_raw["is_open"].isna()].shape

In [ ]:
business_raw[business_raw["latitude"].isna()].shape

In [ ]:
business_raw[business_raw["longitude"].isna()].shape

In [ ]:
business_raw[business_raw["name"].isna()].shape

In [ ]:
business_raw[business_raw["postal_code"].isna()].shape

In [ ]:
business_raw[business_raw["review_count"].isna()].shape

In [ ]:
business_raw[business_raw["stars"].isna()].shape

In [ ]:
business_raw[business_raw["state"].isna()].shape

In [ ]:
business_raw[business_raw["is_open"] == 1][business_raw["hours"].isna()]

In [ ]:
business_raw[business_raw["postal_code"].isna()][business_raw["address"].isna() == False]

We have 0 business without a state, 1 business without a city, 659 businesses without a postal_code, 7682 businesses without an address among which only 545 has no postal_code, so we have 114 businesses with an address but no postal_code

In [ ]:
eval(business_raw["attributes"][2])


# Parse businesses ids

In [ ]:
business_ids = business_raw["business_id"].reset_index().set_index("business_id")["index"].to_dict()
del business_raw["business_id"]
business_table = business_raw.reset_index().rename(columns={'index':'id'})
business_table.head()

# Parse Locations

In [ ]:
locations = business_table[["id", "address", "city", "latitude", "longitude", "postal_code", "state"]]
del business_table["address"]
del business_table["city"]
del business_table["latitude"]
del business_table["longitude"]
del business_table["postal_code"]
del business_table["state"]
locations = locations.rename(columns={'id':'business_id'})
locations.head()

In [ ]:
locations.head()

# Parse states

In [ ]:
states = locations["state"].unique()
states.sort()
states

In [ ]:
state_table = pd.Series(states, name="state")
state_table.index += 1
state_table = state_table.reset_index().rename(columns={"index":"id"})
state_table.head()

In [ ]:
state_dict = state_table.set_index("state")["id"].to_dict()
locations["state"] = locations["state"].apply(lambda s: state_dict[s])
locations = locations.rename(columns={'state':'state_id'})
locations.head()

# Parse cities

In [ ]:
city_table = locations[["city", "state_id"]].drop_duplicates()
city_table.index += 1
city_table = city_table.reset_index().rename(columns={"index":"id", "city":"name"})
city_table.head()

In [ ]:
city_dict = city_table.set_index(["name", "state_id"])["id"].to_dict()

In [ ]:
locations["city_id"] = locations[["city", "state_id"]].apply(lambda row: city_dict[(row[0], row[1])], axis=1, raw=True)
del locations["state_id"]
del locations["city"]
locations.head()

# Parse postal codes

In [ ]:
postal_codes_table = locations[["city_id", "postal_code"]].drop_duplicates()
postal_codes_table.index += 1
postal_codes_table = postal_codes_table.reset_index().rename(columns={"index":"id"})
postal_codes_table.head()

In [ ]:
postal_code_dict = postal_codes_table.set_index(["postal_code", "city_id"])["id"].to_dict()

In [ ]:
locations["postal_code_id"] = locations[["postal_code", "city_id"]].apply(lambda row: postal_code_dict[(row[0], row[1])], axis=1, raw=True)
del locations["city_id"]
del locations["postal_code"]
locations.head()

# Parse categories

In [ ]:
business_categories_temp = business_table[["id", "categories"]][business_table["categories"].map(lambda x: type(x) == str)]
business_categories_temp["categories"] = business_categories_temp["categories"].progress_map(lambda cats: list(map(str.strip, cats.split(","))))

business_categories_temp.head()

In [ ]:
business_categories = (business_categories_temp['categories']
    .progress_apply(lambda x: pd.Series(x))
    .stack()
    .reset_index(level=1, drop=True)
    .to_frame('categories')
    .join(business_categories_temp[['id']], how='left'))
business_categories

In [ ]:
categories = business_categories["categories"].reset_index()
del categories["index"]
categories = categories.drop_duplicates()
categories.index += 1
categories = categories.reset_index().rename(columns={"index":"id", "categories": "name"})
categories

In [ ]:
categories_dict = categories.reset_index().set_index("name")["index"].to_dict()
business_categories["categorie_id"] = business_categories["categories"].progress_map(lambda cat: categories_dict[cat])
del business_categories["categories"]
business_categories

In [ ]:
business_table.head()

# Store in data base

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

# stores states
for name, id in state_dict.items():
    cur.execute("insert into state (id, name) values ({id}, '{name}')".format(id=id, name=name))
con.commit()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")